In [1]:
import re
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
load_dotenv()
chat = ChatGoogleGenerativeAI(temperature=0.0, model="gemini-2.0-flash")

In [2]:
class Agent:
  def __init__(self, llm_model="gemini-2.0-flash", system=""):
    self.system = system
    self.messages = []
    if self.system:
      self.messages.append(("system", self.system))
      self.chat = ChatGoogleGenerativeAI(model=llm_model, temperature=0.0)

  def __call__(self, message):
    self.messages.append(("user", message))
    result = self.execute()
    self.messages.append(("assistant", result))
    return result

  def execute(self):
    ai_msg = self.chat.invoke(self.messages)
    return ai_msg.content

In [15]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.
Use Thought to describe your thoughts about the question.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:
movie_plot:
e.g. movie_plot: Inception
Returns the summary plot of the movie

character_count:
e.g. character_count: Hello world
Returns the number of characters in the input string

Example session:
Question: What is the plot of Titanic?
Thought: I should get the plot of the movie Titanic
Action: movie_plot: Titanic
PAUSE

You will be called again with this:
Observation: Titanic is a romantic drama about a couple on the ill-fated RMS Titanic.

You then output:
Answer: Titanic is a romantic drama about a couple on the ill-fated RMS Titanic.

Note: You should not use your own brain to answer if you have available action to execute the same.
""".strip()

In [16]:
def movie_plot(title):
  plots = {
  "Titanic": "Titanic is a romantic drama about a couple on the ill-fated RMS Titanic.",
  "Inception": "Inception is a sci-fi film where dream invasion and manipulation are possible.",
  "The Matrix": "The Matrix is about a man who discovers reality is a simulated illusion."
  }
  return plots.get(title.strip(), "Sorry, I don't know the plot for that movie.")
def character_count(text):
  return f"The input has {len(text)} characters."

In [17]:
action_re = re.compile('^Action: (\w+): (.*)$')

known_actions = {
"movie_plot": movie_plot,
"character_count": character_count
}

def query(question, max_turns=5):
  i = 0
  bot = Agent(system=prompt)
  next_prompt = question
  while i < max_turns:
    i += 1
    result = bot(next_prompt)
    print(result)
    actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
    if actions:
      action, action_input = actions[0].groups()
      if action not in known_actions:
        raise Exception(f"Unknown action: {action}: {action_input}")
      print(f" - running {action}: {action_input}")
      observation = known_actions[action](action_input)
      print("Observation:", observation)
      next_prompt = f"Observation: {observation}"
    else:
      return

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
/var/folders/c9/h7x3gp7x57n7kz8n91yjhw6w0000gn/T/ipykernel_63644/1435635788.py:1: SyntaxWarning: invalid escape sequence '\w'
  action_re = re.compile('^Action: (\w+): (.*)$')


In [18]:
query("What is the plot of Inception?")

Thought: I should use the movie_plot tool to find the plot of Inception.
Action: movie_plot: Inception
PAUSE
 - running movie_plot: Inception
Observation: Inception is a sci-fi film where dream invasion and manipulation are possible.
Answer: Inception is a sci-fi film where dream invasion and manipulation are possible.


In [19]:
query("How many characters are in the sentence 'Life is beautiful'?")

Thought: I need to count the number of characters in the sentence 'Life is beautiful'.
Action: character_count: Life is beautiful
PAUSE
 - running character_count: Life is beautiful
Observation: The input has 17 characters.
Answer: The sentence 'Life is beautiful' has 17 characters.


In [20]:
query("What is the plot of Inception? also, how many charachter does the plot have?")

Thought: I need to find the plot of the movie Inception and then count the number of characters in the plot.
Action: movie_plot: Inception
PAUSE
 - running movie_plot: Inception
Observation: Inception is a sci-fi film where dream invasion and manipulation are possible.
Thought: Now that I have the plot of Inception, I need to count the number of characters in the plot.
Action: character_count: Inception is a sci-fi film where dream invasion and manipulation are possible.
PAUSE
 - running character_count: Inception is a sci-fi film where dream invasion and manipulation are possible.
Observation: The input has 78 characters.
Answer: Inception is a sci-fi film where dream invasion and manipulation are possible. The plot has 78 characters.
